In [1]:
import os.path as op, os
import nibabel as nib
import numpy as np
import pandas as pd

from copy import deepcopy

from skimage.measure import compare_ssim as ssim
from skimage.measure import compare_mse as mse
from skimage.measure import compare_nrmse as nrmse
from skimage.measure import compare_psnr as psnr

In [2]:
def oneVoxel(data, loc, scale):
    loc = tuple(loc)
    assert(len(loc) == len(data.shape))
    datnew = np.copy(data)
    datnew[loc] = datnew[loc] * (scale)
    return datnew

In [13]:
diff_df = pd.DataFrame(columns = ["Images", "SameSubject", "1-voxel", "Rician", "Gaussian", "MSE", "NRMSE", "PSNR", "SSIM"])
image_df = pd.DataFrame(columns = ["Dataset", "Subject", "Session", "Modality", "Noise", "Affine", "Image"])

In [14]:
f_image1 = '/data/hcp1200_minimal/sub-100206/ses-1/anat/sub-100206_ses-1_T1w.nii.gz'
f_image2 = '/data/hcp1200_minimal/sub-108121/ses-1/anat/sub-108121_ses-1_T1w.nii.gz'

im1 = nib.load(f_image1)
im2 = nib.load(f_image2)

dat1 = im1.get_data()
dat2 = im2.get_data()

image_df.loc[len(image_df)] = ["HCP", "100206", "1", "T1w", None, im1.affine, dat1]
image_df.loc[len(image_df)] = ["HCP", "108121", "1", "T1w", None, im2.affine, dat2]

In [15]:
noisyim = deepcopy(image_df.loc[0][:])
loc = (150, 160, 160)
params = [noisyim[-1], loc, 1.15]
noisyim[4] = "fn: {0}; params: {1}".format('oneVoxel', str(params[1:]))
noisyim[6] = oneVoxel(*params)
image_df.loc[len(image_df)] = noisyim

noisyim = deepcopy(image_df.loc[1][:])
params = [noisyim[-1], loc, 1.15]
noisyim[4] = "fn: {0}; params: {1}".format('oneVoxel', str(params[1:]))
noisyim[6] = oneVoxel(*params)
image_df.loc[len(image_df)] = noisyim

In [16]:
image_df[image_df.columns[:-2]]

,Dataset,Subject,Session,Modality,Noise
0,HCP,100206,1,T1w,None
1,HCP,108121,1,T1w,None
2,HCP,100206,1,T1w,"fn: oneVoxel; params: [(150, 160, 160), 1.15]"
3,HCP,108121,1,T1w,"fn: oneVoxel; params: [(150, 160, 160), 1.15]"


In [18]:
for idx, im1 in image_df.iterrows():
    for jdx, im2 in image_df.iterrows():
        if jdx <= idx:
            continue
        diff_df.loc[len(diff_df)] = [(1, 2),  # Image indices
                                     im1[1] == im2[1],  # Same subject?
                                     ("oneVoxel" in [im1[4] or im2[4]]),  # One voxel noise?
                                     False,  # Rician noise?
                                     False,  # Gaussian noise?
                                     mse(im1[-1], im2[-1]),
                                     nrmse(im1[-1], im2[-1]),
                                     psnr(im1[-1], im2[-1]),
                                     ssim(im1[-1], im2[-1])]

ValueError: cannot set a row with mismatched columns

In [ ]:
diff_df

In [12]:
diff_df.loc[1][-1], diff_df.loc[4][-1]

(0.9999999999450607, 0.9999999999004779)